In [1]:
# Add directory above current directory to path
import sys; sys.path.insert(0, '..')

from functools import partial
from pathlib import Path
from pprint import pprint

import optuna
import pandas as pd
from IPython.display import display

from config.convolutional_features import *
from data_preparation.io_utils import read_yaml_file
from dataset.dataset_builder import DatasetBuilder
from model.model_builder import ModelBuilder
from model.model_tuning import OptunaOptimizer
from pipeline.trainer import Trainer

CONFIG_PATH = Path('../config/config.yaml')

In [2]:
# Data Configuration
config = read_yaml_file(CONFIG_PATH)
data_config = config['data']
print(data_config)
conv_pattern = "**/convolutional.csv"

{'data_dir': 'training_data', 'test_models': ['lenet', 'resnet18', 'vgg16']}


In [3]:
dataset_builder = DatasetBuilder(features=CONV_FEATURES)
conv_dataset = dataset_builder.create_dataset(
            data_dir=Path('../training_data'),
            test_models=data_config["test_models"],
            pattern=conv_pattern,
        )

In [4]:
print(f"Number of training samples: {len(conv_dataset.train.input_features)}")
print(f"Number of testing samples: {len(conv_dataset.test.input_features)}")

Number of training samples: 550
Number of testing samples: 35


## Baseline Models


Baseline models for power and runtime models, we will use the mean of train dataset as prediction.


### Baseline Result

Test dataset RMSPE Power : 85.87%

Test dataset RMSPE Runtime: 110.50%

In [5]:
trainer = Trainer(data_config=data_config, model_config=config['model'], features=CONV_FEATURES)

In [6]:
mean_power = conv_dataset.train.power.mean()
target = conv_dataset.test.power

In [7]:
pprint(trainer.eval_metrics(actual=target, pred=[mean_power]*len(target)))

{'testing_mean_absolute_error': np.float64(1.7149496159728483),
 'testing_mean_absolute_percentage_error': np.float64(0.44269584130221895),
 'testing_mean_squared_error': np.float64(3.8481622171144707),
 'testing_r2_score': -0.007932728803180078,
 'testing_root_mean_squared_error': np.float64(1.9616733206919217),
 'testing_root_mean_squared_percentage_error': np.float64(85.87328424670017)}


In [8]:
mean_runtime = conv_dataset.train.runtime.mean()
target = conv_dataset.test.runtime

In [9]:
pprint(trainer.eval_metrics(actual=target, pred=[mean_runtime]*len(target)))

{'testing_mean_absolute_error': np.float64(0.12056363355248609),
 'testing_mean_absolute_percentage_error': np.float64(0.8375628311081222),
 'testing_mean_squared_error': np.float64(0.037841535688586815),
 'testing_r2_score': -0.22620085772044263,
 'testing_root_mean_squared_error': np.float64(0.1945290098894939),
 'testing_root_mean_squared_percentage_error': np.float64(110.50456893679386)}


## Power

[Optuna](https://optuna.org/) library is used to perform hyperparameter tuning for power model.

In [10]:
X_train = conv_dataset.train.input_features.values
y_power_train = conv_dataset.train.power.values
print(f"Training shape: {X_train.shape}, {y_power_train.shape}")

X_test = conv_dataset.test.input_features.values
y_power_test = conv_dataset.test.power.values
print(f"Testing shape: {X_test.shape}, {y_power_test.shape}")

Training shape: (550, 13), (550,)
Testing shape: (35, 13), (35,)


In [11]:
model_builder = ModelBuilder()
optimizer = OptunaOptimizer(X_train=X_train, y_train=y_power_train, X_test=X_test, y_test=y_power_test, model_builder=model_builder)

In [ ]:
%%time

# Ignore ConvergenceWarning from sklearn to avoid tab crash
from warnings import filterwarnings
filterwarnings('ignore')

# Maximize the test R^2 score during tuning
power_study = optuna.create_study(study_name='conv_power_model_tuning', direction="maximize", storage="sqlite:///conv_power_model_tuning.db")
# Run study for 100 trials
power_study.optimize(partial(optimizer.objective, 
                             features_mapping=dataset_builder.features_mapping, 
                             special_terms_list=[TOTAL_CONV_OPS_PER_INPUT, TOTAL_CONV_OPS_PER_BATCH]), 
               n_trials=100)

[I 2024-11-26 18:01:00,611] A new study created in RDB with name: conv_power_model_tuning
[I 2024-11-26 18:01:00,984] Trial 0 finished with value: 0.10253354479278853 and parameters: {'degree': 3, 'log_scale': False, 'special_features': True, 'scalers': 'robust', 'max_iter': 2730, 'n_alphas': 485, 'fit_intercept': True, 'positive': True}. Best is trial 0 with value: 0.10253354479278853.
[I 2024-11-26 18:01:01,579] Trial 1 finished with value: -7.66286511952006 and parameters: {'degree': 2, 'log_scale': True, 'special_features': True, 'scalers': 'standard', 'max_iter': 18417, 'n_alphas': 892, 'fit_intercept': False, 'positive': False}. Best is trial 0 with value: 0.10253354479278853.
[I 2024-11-26 18:02:00,647] Trial 2 finished with value: 0.7385479094333176 and parameters: {'degree': 4, 'log_scale': True, 'special_features': False, 'scalers': 'minmax', 'max_iter': 1187, 'n_alphas': 137, 'fit_intercept': False, 'positive': True}. Best is trial 2 with value: 0.7385479094333176.


In [ ]:
# print the best performing pipeline
pprint(power_study.best_trial)

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(power_study)

## Runtime

[Optuna](https://optuna.org/) library is used to perform hyperparameter tuning for runtime model.

In [ ]:
X_train = conv_dataset.train.input_features.values
y_runtime_train = conv_dataset.train.runtime.values
print(f"Training shape: {X_train.shape}, {y_runtime_train.shape}")

X_test = conv_dataset.test.input_features.values
y_runtime_test = conv_dataset.test.runtime.values
print(f"Testing shape: {X_test.shape}, {y_runtime_test.shape}")

In [ ]:
model_builder = ModelBuilder()
optimizer = OptunaOptimizer(X_train=X_train, y_train=y_runtime_train, X_test=X_test, y_test=y_runtime_test, model_builder=model_builder)

In [ ]:
%%time

# Ignore ConvergenceWarning from sklearn to avoid tab crash
from warnings import filterwarnings
filterwarnings('ignore')

# Maximize the test R^2 score during tuning
runtime_study = optuna.create_study(study_name='conv_runtime_model_tuning', direction="maximize", storage="sqlite:///conv_runtime_model_tuning.db")
# Run study for 100 trials
runtime_study.optimize(partial(optimizer.objective, 
                               features_mapping=dataset_builder.features_mapping, 
                               special_terms_list=[TOTAL_CONV_OPS_PER_INPUT, TOTAL_CONV_OPS_PER_BATCH]), 
               n_trials=100)

In [ ]:
# print the best performing pipeline
pprint(runtime_study.best_trial)

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(runtime_study)